$$
  H = \sum_n \left[ \frac{p_n^2}{2a} + \frac{a}{2}\left(\frac{\phi_{n+1}-\phi_{n-1}}{2a}\right)^2+\frac{a}{2}V(\phi_n)^2 + aV(\phi_n)\frac{\phi_{n+1}-\phi_{n-1}}{2a} \right. \nonumber \\
             \quad \left.+(-1)^nV'(\phi_n)\left(\chi_n^{\dagger}\chi_n-\frac{1}{2}\right) + \frac{1}{2a}\left(\chi_n^{\dagger}\chi_{n+1}+\chi_{n+1}^{\dagger}\chi_n\right) \right], 
$$

# Sparse matrix form

In [1]:
import scipy
from scipy.sparse import eye, kron, coo_matrix
from scipy.sparse.linalg import eigsh

import numpy as np
from susy_qm import create_matrix
from functools import reduce

In [109]:
# Parameters
N = 2
cutoff = 4
a = 1.0

In [110]:
def kron_tensor(size, site, total_sites, operator):

    I = eye(size, format='coo', dtype=np.complex128)
    operators = [I] * total_sites
    operators[site] = operator
  
    return reduce(kron, operators) 

In [ ]:
# Initialize the Hamiltonian as a sparse matrix
I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')
dim = I_b.size * I_f.size
H = coo_matrix((dim, dim), dtype=np.complex128)

# Define operators as sparse matrices
q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

boundary_conditions = 'dirichlet' 
#boundary_conditions ='periodic'

for n in range(N):
    
    q_n = kron_tensor(cutoff, n, N, q)
    p_n = kron_tensor(cutoff, n, N, p)

    chi_n = kron_tensor(2, n, N, chi)
    chidag_n = kron_tensor(2, n, N, chidag)
    

    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  
    kinetic_term = kron(p2, I_f, format='coo') 

    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1
    potential = coo_matrix(W_prime @ W_prime * (a / 2))
    potential_term = kron(potential, I_f, format='coo')

    # Fermionic term
    commutator_term = ((-1) ** n) * kron(W_double_prime, coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')

    H += kinetic_term + potential_term + commutator_term 
    
    # Cross-site terms
    if boundary_conditions == 'dirichlet':
        print('dirichlet')
        # Set operators to zero matrices at boundary
        zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
        zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

        # Dirichlet BC's
        if n < N-1:
            print(n)
            q_np1 = kron_tensor(cutoff, (n + 1), N, q)
            chi_np1 = kron_tensor(2, (n + 1), N, chi)
            chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
        else:
            print(n, 'Zero')
            q_np1 = zero_qop
            chi_np1 = zero_cop
            chidag_np1 = zero_cop

        if n == 0:
            q_nm1 = zero_qop
        else:
            q_nm1 = kron_tensor(cutoff, (n - 1), N, q)
    
    elif boundary_conditions == 'periodic':
        print('periodic')
        q_np1 = kron_tensor(cutoff, (n + 1) % N, N, q)
        q_nm1 = kron_tensor(cutoff, (n - 1) % N, N, q)
        chi_np1 = kron_tensor(2, (n + 1) % N, N, chi)
        chidag_np1 = kron_tensor(2, (n + 1) % N, N, chidag)


    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = kron((a / 2) * (gradient @ gradient), I_f, format='coo')
    potential_gradient_term = kron(a * (W_prime @ gradient), I_f, format='coo')

    # Fermionic hopping
    fermion = coo_matrix((chidag_n @ chi_np1 + chidag_np1 @ chi_n)) / (2 * a)
    #print(fermion)
    fermion_term = kron(I_b, fermion, format='coo')
    constant_offset = (N-n / (2 * a))
    fermion_term = fermion_term + constant_offset * eye(H.shape[0], format='coo')

    H += potential_gradient_term + fermion_term


dirichlet
0
dirichlet
1 Zero


In [118]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 6
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')

print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [0.38196601 1.38196601 1.38196601 1.38196601 1.38196601 1.5       ]


In [81]:
np.sort(eigenvalues +0.44090964)#+ 0.41421356)

array([-4.28219166e-09,  9.99999996e-01,  9.99999996e-01,  9.99999996e-01,
        9.99999996e-01,  9.99999996e-01])

In [248]:
# Parameters
N = 2
cutoff = 16
a = 1.0

In [249]:
np.set_printoptions(threshold = np.inf)

In [250]:
# Initialize the Hamiltonian as a sparse matrix
I_b = eye(cutoff ** N, format='coo')
I_f = eye(2 ** N, format='coo')
dim = I_b.size * I_f.size
H = coo_matrix((dim, dim), dtype=np.complex128)

# Define operators as sparse matrices
q = coo_matrix(create_matrix(cutoff, 'q'))
p = coo_matrix(create_matrix(cutoff, 'p'))
chi = coo_matrix([[0, 1], [0, 0]], dtype=np.complex128)
chidag = coo_matrix([[0, 0], [1, 0]], dtype=np.complex128)

boundary_conditions = 'dirichlet' 
#boundary_conditions ='periodic'

for n in range(N):
    
    q_n = kron_tensor(cutoff, n, N, q)
    p_n = kron_tensor(cutoff, n, N, p)

    chi_n = kron_tensor(2, n, N, chi)
    chidag_n = kron_tensor(2, n, N, chidag)
    

    # Kinetic term
    p2 = coo_matrix(p_n @ p_n / (2 * a))  
    kinetic_term = kron(p2, I_f, format='coo') 

    # Potential term
    W_prime = q_n  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1
    potential = coo_matrix(W_prime @ W_prime * (a / 2))
    potential_term = kron(potential, I_f, format='coo')

    # Fermionic term
    commutator_term = ((-1) ** n) * kron(W_double_prime, coo_matrix(chidag_n @ chi_n - 0.5 * I_f), format='coo')

    H += kinetic_term + potential_term + commutator_term 
    
    # Cross-site terms
    if boundary_conditions == 'dirichlet':
    
        # Set operators to zero matrices at boundary
        zero_qop = coo_matrix((I_b.size,I_b.size), dtype=np.complex128)
        zero_cop = coo_matrix((I_f.size,I_f.size), dtype=np.complex128)

        # Dirichlet BC's
        if n < N-1:
          
            q_np1 = kron_tensor(cutoff, (n + 1), N, q)
            chi_np1 = kron_tensor(2, (n + 1), N, chi)
            chidag_np1 = kron_tensor(2, (n + 1) , N, chidag)
        else:
      
            q_np1 = zero_qop
            chi_np1 = zero_cop
            chidag_np1 = zero_cop

        if n == 0:
            q_nm1 = zero_qop
        else:
            q_nm1 = kron_tensor(cutoff, (n - 1), N, q)
    
    elif boundary_conditions == 'periodic':

        q_np1 = kron_tensor(cutoff, (n + 1) % N, N, q)
        q_nm1 = kron_tensor(cutoff, (n - 1) % N, N, q)
        chi_np1 = kron_tensor(2, (n + 1) % N, N, chi)
        chidag_np1 = kron_tensor(2, (n + 1) % N, N, chidag)


    gradient = coo_matrix((q_np1 - q_nm1) / (2 * a))
    gradient_term = kron((a / 2) * (gradient @ gradient), I_f, format='coo')
    potential_gradient_term = kron(a * (W_prime @ gradient), I_f, format='coo')

    # Fermionic hopping
    fermion = coo_matrix((chidag_n @ chi_np1 + chidag_np1 @ chi_n)) / (2 * a)
    #print(fermion)
    fermion_term = kron(fermion, I_b, format='coo')
    #fermion_term = kron(I_b, fermion, format='coo')

    H += potential_gradient_term + fermion_term


In [251]:
# Compute the k smallest eigenvalues of the sparse Hamiltonian
num_eigenvalues = 6
eigenvalues, eigenvectors = eigsh(H, k=num_eigenvalues, which='SA')

print("Eigenvalues:", np.sort(eigenvalues))

Eigenvalues: [2.80767281e-15 1.00000000e+00 1.00000000e+00 1.00000000e+00
 1.00000000e+00 2.00000000e+00]


In [224]:
(chidag_n @ chi_np1).todense()

matrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [225]:
(chidag_np1 @ chi_n).todense()

matrix([[0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
        [0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]])

In [227]:
print(kron(p2, I_f, format='coo') )

<COOrdinate sparse matrix of dtype 'complex128'
	with 2816 stored elements and shape (1024, 1024)>
  Coords	Values
  (0, 128)	(-0.35355339059327384+0j)
  (1, 129)	(-0.35355339059327384+0j)
  (2, 130)	(-0.35355339059327384+0j)
  (3, 131)	(-0.35355339059327384+0j)
  (0, 0)	(0.25000000000000006+0j)
  (1, 1)	(0.25000000000000006+0j)
  (2, 2)	(0.25000000000000006+0j)
  (3, 3)	(0.25000000000000006+0j)
  (4, 132)	(-0.35355339059327384+0j)
  (5, 133)	(-0.35355339059327384+0j)
  (6, 134)	(-0.35355339059327384+0j)
  (7, 135)	(-0.35355339059327384+0j)
  (4, 4)	(0.25000000000000006+0j)
  (5, 5)	(0.25000000000000006+0j)
  (6, 6)	(0.25000000000000006+0j)
  (7, 7)	(0.25000000000000006+0j)
  (8, 136)	(-0.35355339059327384+0j)
  (9, 137)	(-0.35355339059327384+0j)
  (10, 138)	(-0.35355339059327384+0j)
  (11, 139)	(-0.35355339059327384+0j)
  (8, 8)	(0.25000000000000006+0j)
  (9, 9)	(0.25000000000000006+0j)
  (10, 10)	(0.25000000000000006+0j)
  (11, 11)	(0.25000000000000006+0j)
  (12, 140)	(-0.35355339059